# Data Understanding
## Overview
Using the data we have, we will attempt to gain more insight on how to construct the best model that predicts how waterpoints in the focus region that is the country: Tanzania are designed.
In this phase of the investigation, the study inspected the data in an attempt to seek a deeper understanding about the data.

## Data Description
The target variable is ``status_group`` with the labels:
* functional - the waterpoint is operational and there are no repairs needed
* functional needs repair - the waterpoint is operational, but needs repairs
* non functional - the waterpoint is not operational

The predictor variables in this data include:
* amount_tsh - Total static head (amount water available to waterpoint)
* date_recorded - The date the row was entered
* funder - Who funded the well
* gps_height - Altitude of the well
* installer - Organization that installed the well
* longitude - GPS coordinate
* latitude - GPS coordinate
* wpt_name - Name of the waterpoint if there is one
* num_private -
* basin - Geographic water basin
* subvillage - Geographic location
* region - Geographic location
* region_code - Geographic location (coded)
* district_code - Geographic location (coded)
* lga - Geographic location
* ward - Geographic location
* population - Population around the well
* public_meeting - True/False
* recorded_by - Group entering this row of data
* scheme_management - Who operates the waterpoint
* scheme_name - Who operates the waterpoint
* permit - If the waterpoint is permitted
* construction_year - Year the waterpoint was constructed
* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses
* management - How the waterpoint is managed
* management_group - How the waterpoint is managed
* payment - What the water costs
* payment_type - What the water costs
* water_quality - The quality of the water
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint

In [198]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import scipy.stats

# Import visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

plt.style.use('ggplot')

import warnings
warnings.filterwarnings("ignore")

In [199]:
# Loading the training_set_values
training_data = pd.read_csv('data/training_set_values.csv')

# Loading the test set values
testing_data = pd.read_csv('data/test_set_values.csv')

# Loading the target column
training_labels = pd.read_csv('data/training_set_labels.csv')

# concatenating train and test data
#training_labels_df = pd.concat([training_data, training_labels],axis=1)

# merging the data sets
training_labels_df = training_data.merge(training_labels, how = "inner")

# previewing the data set
training_labels_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,3/14/2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,3/6/2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2/25/2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,1/28/2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,7/13/2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [200]:
# previewing the last five rows of the data
training_labels_df.tail()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
59395,60739,10.0,5/3/2013,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,5/7/2011,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,4/11/2011,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,3/8/2011,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional
59399,26348,0.0,3/23/2011,World Bank,191,World,38.104048,-6.747464,Kwa Mzee Lugawa,0,...,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [201]:
# getting the shape of the data
print(f" This data has {training_labels_df.shape[0]} rows and {training_labels_df.shape[1]} columns")

 This data has 59400 rows and 41 columns


### Preliminary Data Inspection

In [202]:
 # Getting the 'data' about the data
training_labels_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [203]:
# Obtaining the summary descriptive statistics of the data
training_labels_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,59400.0,37115.131768,21453.128371,0.00000,18519.750000,37061.500000,55656.500000,7.424700e+04
amount_tsh,59400.0,317.650385,2997.574558,0.00000,0.000000,0.000000,20.000000,3.500000e+05
gps_height,59400.0,668.297239,693.116350,-90.00000,0.000000,369.000000,1319.250000,2.770000e+03
longitude,59400.0,34.077427,6.567432,0.00000,33.090347,34.908743,37.178387,4.034519e+01
latitude,59400.0,-5.706033,2.946019,-11.64944,-8.540621,-5.021597,-3.326156,-2.000000e-08
num_private,59400.0,0.474141,12.236230,0.00000,0.000000,0.000000,0.000000,1.776000e+03
region_code,59400.0,15.297003,17.587406,1.00000,5.000000,12.000000,17.000000,9.900000e+01
district_code,59400.0,5.629747,9.633649,0.00000,2.000000,3.000000,5.000000,8.000000e+01
population,59400.0,179.909983,471.482176,0.00000,0.000000,25.000000,215.000000,3.050000e+04
construction_year,59400.0,1300.652475,951.620547,0.00000,0.000000,1986.000000,2004.000000,2.013000e+03


#### Observations
``region_code`` and ``district_code`` seem like they should be categorical features.

``construction_year`` should be casted as a datetime object.

``id`` should be casted to object.

``longitude`` and ``latitude`` look fine.

``gps_height``, ``amount_tsh`` and ``population`` also look fine.

In [204]:
# Getting the data types of the data
training_labels_df.dtypes.value_counts()

object     31
int64       7
float64     3
dtype: int64

There are 10 numeric features in the data and 31 string features in the data. 

In [205]:
# getting the value counts of num_private
training_labels_df.num_private.value_counts()

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

In [206]:
# getting the value counts of manegement
training_labels_df.management.value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [207]:
# getting the value counts of management_group
training_labels_df.management_group.value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

In [208]:
# getting the value counts of scheme_management
training_labels_df.scheme_management.value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [209]:
# Getting the value counts of scheme_name
training_labels_df.scheme_name.value_counts()

K                       682
None                    644
Borehole                546
Chalinze wate           405
M                       400
                       ... 
Mafuriko  Water Supp      1
Chusuro water sup         1
BL Sia Eki                1
Kaembe project            1
Magula Mwanyumba          1
Name: scheme_name, Length: 2696, dtype: int64

In [210]:
# # getting the value counts of payment_type
training_labels_df.payment_type.value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

In [211]:
# getting the value counts of payment
training_labels_df.payment.value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [212]:
# Getting the value counts of source
training_labels_df.source.value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [213]:
# # getting the value counts of source_class
training_labels_df.source_class.value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [214]:
irrelevant_columns = ['date_recorded', 'num_private', 'wpt_name',
                     'subvillage', 'region_code', 'district_code', 'lga',
                     'ward', 'public_meeting', 'recorded_by', 'scheme_management',
                     'scheme_name', 'extraction_type', 'extraction_type_group', 
                      'water_quality', 'source','source_type', 'waterpoint_type_group',
                     'payment_type', 'management', 'quantity_group']

In [215]:
print(f" There are {irrelevant_columns.__len__()} irrelevant columns in the data")

 There are 21 irrelevant columns in the data


After going through the variable description of the data and performing the preliminary data inspection,
the study has proposed that the columns categorized as **irrelevant_columns** be dropped on the basis that some provide similar information and some do not provide any relevant information, such as ``public_meeting`` and ``num_private``.

# Data Preparation
## Overview
It is vital for data to be prepared before being staged for modelling to enhance the model's efficiency and prevent the generation of misleading knowledge.
In this phase of the investigation, the study will look at missing values, duplicated entries, inconsistencies and invalid data.

In [216]:
# dropping irrelevant columns
data = training_labels_df.drop(irrelevant_columns, axis=1)

# previewing the new data
data.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,permit,construction_year,extraction_type_class,management_group,payment,quality_group,quantity,source_class,waterpoint_type,status_group
0,69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,109,False,1999,gravity,user-group,pay annually,good,enough,groundwater,communal standpipe,functional
1,8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,280,True,2010,gravity,user-group,never pay,good,insufficient,surface,communal standpipe,functional
2,34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Pangani,Manyara,250,True,2009,gravity,user-group,pay per bucket,good,enough,surface,communal standpipe multiple,functional
3,67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,58,True,1986,submersible,user-group,never pay,good,dry,groundwater,communal standpipe multiple,non functional
4,19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,0,True,0,gravity,other,never pay,good,seasonal,surface,communal standpipe,functional


In [217]:
# getting the shape of the new data
print(f"The new data has {data.shape[0]} rows and {data.shape[1]} columns")

The new data has 59400 rows and 20 columns


The **irrelevant_columns** were successfully dropped.

## Data Cleaning

### Checking for and handling duplicates

In [218]:
# Checking for duplicated entries
data.id.duplicated().sum()

0

This data does not have duplicated entries on the unique identifier ``id``.

### Checking for and handling Missing values

In [219]:
# checking for missing values
data.isna().sum()

id                          0
amount_tsh                  0
funder                   3635
gps_height                  0
installer                3655
longitude                   0
latitude                    0
basin                       0
region                      0
population                  0
permit                   3056
construction_year           0
extraction_type_class       0
management_group            0
payment                     0
quality_group               0
quantity                    0
source_class                0
waterpoint_type             0
status_group                0
dtype: int64

In [220]:
#checking for the percentage of missing values of data points in each column
percentages = (data.isna().sum() / len(data)) * 100
percentages.sort_values(ascending=False)

installer                6.153199
funder                   6.119529
permit                   5.144781
status_group             0.000000
region                   0.000000
amount_tsh               0.000000
gps_height               0.000000
longitude                0.000000
latitude                 0.000000
basin                    0.000000
population               0.000000
waterpoint_type          0.000000
construction_year        0.000000
extraction_type_class    0.000000
management_group         0.000000
payment                  0.000000
quality_group            0.000000
quantity                 0.000000
source_class             0.000000
id                       0.000000
dtype: float64

The columns ``funder``, ``installer`` and ``permit`` are the only features with missing values. In addition, the percentage of entries classified as missing values in ``funder`` and ``installer`` are small enough for us to drop those entries without sacrificing a big chunk of our data.

In [221]:
# Having a glimpe of the feature funder
for col in ['funder', 'installer', 'permit']:
    print(f"For column {col}:")
    print(f"There are {data[col].nunique()} unique values in this column")
    print("------------------")

For column funder:
There are 1897 unique values in this column
------------------
For column installer:
There are 2145 unique values in this column
------------------
For column permit:
There are 2 unique values in this column
------------------


In [222]:
# creating an array of densities to impute missing values for permit
densities = list(data.permit.value_counts(normalize=True))

# previewing densities
densities

[0.6895499077097828, 0.31045009229021725]

In [223]:
#setting seed for reproducibility
np.random.seed(0)

# Defining a function to fill missing values for waterfront
def impute_missing(value):
    '''A function that fills missing values for waterfront'''
    if value not in [0,1]:
        return np.random.choice([0,1], p=densities)
    else:
        return value
    
# Applying the function to the column permit    
data['permit'] = data['permit'].map(lambda x: impute_missing(x))

# #previewing the columns to check for missing values
for col in ['funder', 'installer', 'permit']:
    print('For', col,':')
    print('The densities are:', data[col].value_counts(normalize=True).\
        sort_values(ascending=False).head())
    print("Number of missing values are : {}".format(data[col].isna().sum()))
    print("-----------------------------")

For funder :
The densities are: Government Of Tanzania    0.162898
Danida                    0.055841
Hesawa                    0.039487
Rwssp                     0.024639
World Bank                0.024191
Name: funder, dtype: float64
Number of missing values are : 3635
-----------------------------
For installer :
The densities are: DWE           0.312171
Government    0.032738
RWE           0.021634
Commu         0.019015
DANIDA        0.018836
Name: installer, dtype: float64
Number of missing values are : 3655
-----------------------------
For permit :
The densities are: True     0.670657
False    0.329343
Name: permit, dtype: float64
Number of missing values are : 0
-----------------------------


The missing values from ``permit`` have been imputed successfully using a user-defined function based on the probability densities of the feature. But the columns ``installer`` and ``funder`` still have missing values due to the fact that they are string values and the study thought it wise to remove those entries.

In [224]:
# Dropping missing values from  installer and funder along the rows
data.dropna(axis=0, inplace=True)

# Checking once more for missing values
data.isna().sum()

id                       0
amount_tsh               0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
basin                    0
region                   0
population               0
permit                   0
construction_year        0
extraction_type_class    0
management_group         0
payment                  0
quality_group            0
quantity                 0
source_class             0
waterpoint_type          0
status_group             0
dtype: int64

There are no more missing values in the data.

### Type casting invalid data formats to correct formats

In [225]:
#converting the data type of construction_yr to pd.datetime and creating multiple columns out of it
# convert construction_year column from string to datetime
data['construction_year'] = pd.to_datetime(data['construction_year'])
data.construction_year.dtype

dtype('<M8[ns]')

In [226]:
# checking the data types of the columns
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55692 entries, 0 to 59399
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     55692 non-null  int64         
 1   amount_tsh             55692 non-null  float64       
 2   funder                 55692 non-null  object        
 3   gps_height             55692 non-null  int64         
 4   installer              55692 non-null  object        
 5   longitude              55692 non-null  float64       
 6   latitude               55692 non-null  float64       
 7   basin                  55692 non-null  object        
 8   region                 55692 non-null  object        
 9   population             55692 non-null  int64         
 10  permit                 55692 non-null  object        
 11  construction_year      55692 non-null  datetime64[ns]
 12  extraction_type_class  55692 non-null  object        
 13  m

The feature ``construction_year`` has been successfully casted to datetime object.

In [227]:
data.quality_group.value_counts()

good        47900
salty        5130
unknown      1293
milky         785
colored       391
fluoride      193
Name: quality_group, dtype: int64

In [228]:
data.amount_tsh.value_counts()

0.0         38293
500.0        3092
50.0         2342
1000.0       1451
20.0         1433
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 97, dtype: int64

In [229]:
data.quantity.value_counts()

enough          31931
insufficient    13933
dry              5778
seasonal         3409
unknown           641
Name: quantity, dtype: int64

In [230]:
data.quality_group.value_counts()

good        47900
salty        5130
unknown      1293
milky         785
colored       391
fluoride      193
Name: quality_group, dtype: int64

In [231]:
data.basin.value_counts()

Lake Victoria              10047
Pangani                     8812
Rufiji                      7196
Internal                    6588
Lake Tanganyika             6400
Wami / Ruvu                 5949
Ruvuma / Southern Coast     4482
Lake Nyasa                  3768
Lake Rukwa                  2450
Name: basin, dtype: int64

In [232]:
data.status_group.value_counts()

functional                 30226
non functional             21589
functional needs repair     3877
Name: status_group, dtype: int64

### Feature Engineering

In [233]:
# Creating a dictionary to be used in type casting status group to a binary column
# The new values will be 0 for non functional, and 1 for functional and 
status_dict = {'functional':1, 'non functional': 0, 'functional needs repair':1}

In [234]:
# Applying the dictionary using the map method of pandas
data['status_group'] = data["status_group"].map(status_dict)

# Checking the type and the values of the column status_group
print('The type is', data.status_group.dtype)
print()
print('The value counts are:') 
print(data.status_group.value_counts())

The type is int64

The value counts are:
1    34103
0    21589
Name: status_group, dtype: int64


In [236]:
data.status_group.value_counts(normalize=True)

1    0.61235
0    0.38765
Name: status_group, dtype: float64

In [80]:
list(data.population.unique())

[109,
 280,
 250,
 58,
 0,
 1,
 345,
 200,
 35,
 50,
 1000,
 4,
 350,
 210,
 156,
 140,
 260,
 150,
 180,
 70,
 100,
 230,
 30,
 20,
 10,
 45,
 456,
 567,
 130,
 225,
 54,
 75,
 900,
 360,
 544,
 441,
 120,
 40,
 221,
 950,
 1430,
 110,
 256,
 320,
 90,
 804,
 600,
 370,
 305,
 1600,
 400,
 590,
 450,
 80,
 560,
 2500,
 1050,
 36,
 540,
 300,
 12,
 632,
 245,
 59,
 25,
 500,
 570,
 700,
 111,
 630,
 270,
 55,
 1700,
 800,
 87,
 2530,
 86,
 96,
 240,
 203,
 215,
 98,
 160,
 1200,
 309,
 95,
 60,
 85,
 159,
 65,
 48,
 2150,
 1013,
 220,
 1680,
 375,
 2000,
 125,
 3226,
 253,
 88,
 133,
 650,
 163,
 430,
 290,
 2100,
 285,
 425,
 155,
 1500,
 185,
 368,
 115,
 520,
 406,
 970,
 52,
 750,
 380,
 123,
 730,
 105,
 452,
 1320,
 14,
 261,
 82,
 550,
 393,
 348,
 89,
 412,
 530,
 386,
 278,
 212,
 340,
 568,
 418,
 56,
 516,
 6922,
 183,
 6,
 263,
 480,
 1250,
 8,
 321,
 420,
 620,
 5153,
 2353,
 440,
 1015,
 269,
 44,
 174,
 323,
 233,
 258,
 152,
 42,
 358,
 211,
 855,
 1020,
 15,
 69,
 145,

In [ ]:
# what is the relationship between population and: amount_tsh, region, functionality 

In [242]:
# Saving cleaned data set for later use
data.to_csv('./Data/cleaned_data.csv')